In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [2]:
# send the GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:

# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [4]:

Postal_code = []
Borough = []
Neighborhood = []

In [5]:

for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells)>0):
        Postal_code.append(cells[0].text.rstrip('\n'))
        Borough.append(cells[1].text.rstrip('\n'))
        Neighborhood.append(cells[2].text.rstrip('\n'))

In [6]:

# create a new DataFrame from the three lists
toronto_df = pd.DataFrame({"Postal Code": Postal_code,
                           "Borough": Borough,
                           "Neighborhood": Neighborhood})

toronto_df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [7]:
toronto_df.shape

(180, 3)

In [8]:
# drop cells with a borough that is Not assigned
df = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
Postal Code     103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
dtypes: object(3)
memory usage: 2.5+ KB


In [10]:
coordinates = pd.read_csv('https://cocl.us/Geospatial_data')
coordinates.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
neighborhoods = df.merge(coordinates, on="Postal Code", how="left")
neighborhoods.head()
neighborhoods.shape

(103, 5)

In [12]:

address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [13]:
!pip install geopy
!pip install folium


import folium # map rendering library
print("Installed!")

     |████████████████████████████████| 102kB 8.7MB/s ta 0:00:011
Installed!


In [14]:

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [15]:
#select boroughs which is belong to Toronto

borough_names = list(neighborhoods.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['Downtown Toronto', 'East Toronto', 'West Toronto', 'Central Toronto']

In [17]:

toronto_df = neighborhoods[neighborhoods['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(toronto_df.shape)
toronto_df.head()

(39, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [18]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [1]:
#using Foursquare

CLIENT_ID =''
CLIENT_SECRET = ''
VERSION = '' # 
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


In [21]:
radius = 1000
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Postal Code'], toronto_df['Borough'], toronto_df['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [22]:

venues_df = pd.DataFrame(venues)
venues_df.head()
venues_df.columns = ['Postal Code','Borough','Neighborhood','Latitude','Longitude','Venue_name','Venue_lat','Venue_long','Venue_cat']
venues_df.head()
venues_df.shape

(3192, 9)

In [23]:
venues_df.groupby(["Postal Code","Borough","Neighborhood"]).count()
venues_df.shape

(3192, 9)

In [24]:
category = venues_df['Venue_cat'].unique()
category

array(['Bakery', 'Coffee Shop', 'Restaurant', 'Historic Site', 'Park',
       'Distribution Center', 'Farmers Market', 'Breakfast Spot',
       'Chocolate Shop', 'Mediterranean Restaurant', 'Dessert Shop',
       'Spa', 'Performing Arts Venue', 'Liquor Store', 'Tech Startup',
       'Italian Restaurant', 'French Restaurant', 'Café', 'Pub',
       'Gym / Fitness Center', 'Thai Restaurant', 'Pool', 'Shoe Store',
       'Yoga Studio', 'Ice Cream Shop', 'Theater', 'Event Space',
       'Gastropub', 'German Restaurant', 'Furniture / Home Store',
       'Animal Shelter', 'Diner', 'Mexican Restaurant', 'Cosmetics Shop',
       'Art Gallery', 'Asian Restaurant', 'Sandwich Place', 'Food Truck',
       'Gym Pool', 'Grocery Store', 'Middle Eastern Restaurant',
       'Pizza Place', 'Brewery', 'Dance Studio', 'Karaoke Bar',
       'Athletics & Sports', 'Bar', 'Bank', 'Sushi Restaurant',
       'Pet Store', 'Electronics Store', 'Auto Dealership', 'Beer Store',
       'Pakistani Restaurant', 'Indian

In [25]:

len(category)

279

In [26]:
category = venues_df['Venue_cat'].unique()
category

array(['Bakery', 'Coffee Shop', 'Restaurant', 'Historic Site', 'Park',
       'Distribution Center', 'Farmers Market', 'Breakfast Spot',
       'Chocolate Shop', 'Mediterranean Restaurant', 'Dessert Shop',
       'Spa', 'Performing Arts Venue', 'Liquor Store', 'Tech Startup',
       'Italian Restaurant', 'French Restaurant', 'Café', 'Pub',
       'Gym / Fitness Center', 'Thai Restaurant', 'Pool', 'Shoe Store',
       'Yoga Studio', 'Ice Cream Shop', 'Theater', 'Event Space',
       'Gastropub', 'German Restaurant', 'Furniture / Home Store',
       'Animal Shelter', 'Diner', 'Mexican Restaurant', 'Cosmetics Shop',
       'Art Gallery', 'Asian Restaurant', 'Sandwich Place', 'Food Truck',
       'Gym Pool', 'Grocery Store', 'Middle Eastern Restaurant',
       'Pizza Place', 'Brewery', 'Dance Studio', 'Karaoke Bar',
       'Athletics & Sports', 'Bar', 'Bank', 'Sushi Restaurant',
       'Pet Store', 'Electronics Store', 'Auto Dealership', 'Beer Store',
       'Pakistani Restaurant', 'Indian

In [27]:
category.shape

(279,)

In [28]:
len(category)

279

In [ ]:
# since we have taken the frequency we will now take the mean of frequencies and then rank the venue from 1 - 10

toronto_avg = toronto_onehot.groupby(['Postal Code','Borough','Neighborhood']).mean().reset_index()
toronto_avg

In [276]:
toronto_avg[toronto_avg[toronto_avg.columns[3:]]>0].count()

Postal Code                       0
Borough                           0
Neighborhood                      0
Zoo                               1
Accessories Store                 1
Airport                           1
Airport Lounge                    1
American Restaurant              22
Amphitheater                      1
Animal Shelter                    1
Antique Shop                      2
Aquarium                          2
Art Gallery                      20
Art Museum                        4
Arts & Crafts Store              13
Asian Restaurant                  9
Athletics & Sports                4
Auto Dealership                   1
Auto Workshop                     1
BBQ Joint                        10
Bagel Shop                        5
Bakery                           33
Bank                             13
Bar                              21
Baseball Field                    3
Baseball Stadium                  1
Basketball Stadium                6
Beach                       

In [32]:
to_playground = toronto_avg[["Neighborhood","Playground"]]
to_playground

,Neighborhood,Playground
0,The Beaches,0.000000
1,"The Danforth West, Riverdale",0.000000
2,"India Bazaar, The Beaches West",0.000000
3,Studio District,0.000000
4,Lawrence Park,0.000000
5,Davisville North,0.000000
6,North Toronto West,0.000000
7,Davisville,0.000000
8,"Moore Park, Summerhill East",0.031746
9,"Summerhill West, Rathnelly, South Hill, Forest...",0.000000


In [33]:
#clustering
num_clusters = 3
to_clustering = to_playground.drop(["Neighborhood"],1)

In [34]:

# run k-means clustering
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(to_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 0, 0, 0], dtype=int32)

In [35]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
to_merged = to_playground.copy()

# add clustering labels
to_merged["Cluster Labels"] = kmeans.labels_
to_merged

,Neighborhood,Playground,Cluster Labels
0,The Beaches,0.000000,0
1,"The Danforth West, Riverdale",0.000000,0
2,"India Bazaar, The Beaches West",0.000000,0
3,Studio District,0.000000,0
4,Lawrence Park,0.000000,0
5,Davisville North,0.000000,0
6,North Toronto West,0.000000,0
7,Davisville,0.000000,0
8,"Moore Park, Summerhill East",0.031746,1
9,"Summerhill West, Rathnelly, South Hill, Forest...",0.000000,0


In [36]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
to_merged = to_merged.join(venues_df.set_index("Neighborhood"), on="Neighborhood")

print(to_merged.shape)
to_merged.head()

(3192, 11)


,Neighborhood,Playground,Cluster Labels,Postal Code,Borough,Latitude,Longitude,Venue_name,Venue_lat,Venue_long,Venue_cat
0,The Beaches,0.0,0,M4E,East Toronto,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
0,The Beaches,0.0,0,M4E,East Toronto,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
0,The Beaches,0.0,0,M4E,East Toronto,43.676357,-79.293031,Beaches Bake Shop,43.680363,-79.289692,Bakery
0,The Beaches,0.0,0,M4E,East Toronto,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub
0,The Beaches,0.0,0,M4E,East Toronto,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater


In [37]:
all_venues = list(to_merged["Venue_cat"].unique())
competitors = []
for x in all_venues:
    if 'hotel' in x.lower() or 'food' in x.lower() or 'coffee shop' in x.lower() or 'resturant' in x.lower() or 'Other Great Outdoors' in x.lower():
        competitors.append(x)

competitors

['Coffee Shop',
 'Health Food Store',
 'Fast Food Restaurant',
 'Seafood Restaurant',
 'Comfort Food Restaurant',
 'Hotel',
 'Hotel Bar',
 'Food & Drink Shop',
 'Food Truck',
 'Food Court',
 'Food']

In [38]:

playarea= to_merged[to_merged["Venue_cat"].isin(competitors) ].reset_index(drop = True)
playarea.sort_values(["Cluster Labels"],inplace= True)
playarea
fixed_columns = list(playarea.columns[3:5])+list(playarea.columns[0:3])+list(playarea.columns[5:])
playarea = playarea[fixed_columns]

In [39]:
playarea

(392, 11)

In [40]:
#visualize cluster

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [41]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(3)
ys = [i+x+(i*x)**2 for i in range(3)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, venue_name,venue_cat, bor, poi, cluster in zip(playarea['Latitude'], playarea['Longitude'],playarea['Venue_name'],playarea['Venue_cat'], playarea['Borough'], playarea['Neighborhood'], playarea['Cluster Labels']):
    label = folium.Popup('{} ({}): {}, {} - Cluster {}'.format(venue_name, venue_cat,bor, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=2,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [347]:
#examine cluster
#Cluster 0
cluster0 = playarea.loc[playarea['Cluster Labels'] == 0]
print(cluster0.shape[0])
cluster0

362


,Postal Code,Borough,Neighborhood,Playground,Cluster Labels,Latitude,Longitude,Venue_name,Venue_lat,Venue_long,Venue_cat
0,M4E,East Toronto,The Beaches,0.0,0,43.676357,-79.293031,The Remarkable Bean,43.672801,-79.287038,Coffee Shop
250,M5L,Downtown Toronto,Commerce Court / Victoria Hotel,0.0,0,43.648198,-79.379817,Buster's Sea Cove,43.647774,-79.379181,Seafood Restaurant
249,M5L,Downtown Toronto,Commerce Court / Victoria Hotel,0.0,0,43.648198,-79.379817,Delta Hotels by Marriott Toronto,43.642882,-79.383949,Hotel
248,M5L,Downtown Toronto,Commerce Court / Victoria Hotel,0.0,0,43.648198,-79.379817,Balzac's Coffee,43.644373,-79.383065,Coffee Shop
247,M5L,Downtown Toronto,Commerce Court / Victoria Hotel,0.0,0,43.648198,-79.379817,Shangri-La Toronto,43.649129,-79.386557,Hotel
246,M5L,Downtown Toronto,Commerce Court / Victoria Hotel,0.0,0,43.648198,-79.379817,Bulldog On The Block,43.650652,-79.384141,Coffee Shop
245,M5L,Downtown Toronto,Commerce Court / Victoria Hotel,0.0,0,43.648198,-79.379817,M Square Coffee Co,43.651218,-79.383555,Coffee Shop
244,M5L,Downtown Toronto,Commerce Court / Victoria Hotel,0.0,0,43.648198,-79.379817,Versus Coffee,43.651213,-79.375236,Coffee Shop
243,M5L,Downtown Toronto,Commerce Court / Victoria Hotel,0.0,0,43.648198,-79.379817,Starbucks,43.647261,-79.378599,Coffee Shop
242,M5L,Downtown Toronto,Commerce Court / Victoria Hotel,0.0,0,43.648198,-79.379817,John & Sons Oyster House,43.650602,-79.381555,Seafood Restaurant


In [348]:
cluster0.groupby(['Borough','Neighborhood','Venue_cat']).count()

Postal Code  \
Borough          Neighborhood                                       Venue_cat                              
Central Toronto  Davisville                                         Coffee Shop                        8   
                                                                    Fast Food Restaurant               1   
                                                                    Hotel                              1   
                                                                    Seafood Restaurant                 1   
                 Davisville North                                   Coffee Shop                       12   
                                                                    Fast Food Restaurant               3   
                                                                    Food & Drink Shop                  2   
                                                                    Hotel                              1   
                                                                    Seafood Restaurant                 1   
                 Forest Hill North & West                           Coffee Shop                        3   
                                                                    Fast Food Restaurant               1   
                 Lawrence Park                                      Coffee Shop                        1   
                 North Toronto West                                 Coffee Shop                        3   
                                                                    Fast Food Restaurant               1   
                                                                    Food & Drink Shop                  1   
                 Roselawn                                           Coffee Shop                        2   
                 Summerhill West / Rathnelly / South Hill / Fore... Coffee Shop                        7   
                                                                    Hotel                              1   
Downtown Toronto Berczy Park                                        Coffee Shop                       12   
                                                                    Comfort Food Restaurant            1   
                                                                    Food Truck                         1   
                                                                    Hotel                              5   
                                                                    Seafood Restaurant                 2   
                 CN Tower / King and Spadina / Railway Lands / H... Coffee Shop                        2   
                 Central Bay Street                                 Coffee Shop                       10   
                                                                    Fast Food Restaurant               1   
                                                                    Hotel                              2   
                                                                    Seafood Restaurant                 1   
                 Church and Wellesley                               Coffee Shop                       12   
                                                                    Hotel                              1   
                 Commerce Court / Victoria Hotel                    Coffee Shop                       10   
                                                                    Food Truck                         1   
                                                                    Hotel                              5   
                                                                    Seafood Restaurant                 5   
                 First Canadian Place / Underground city            Coffee Shop                        9   
                                                                    Food Truck                   

In [349]:
#Cluster 1
cluster1 = playarea.loc[playarea['Cluster Labels'] == 1]
print(cluster1.shape[0])
cluster1

7


,Postal Code,Borough,Neighborhood,Playground,Cluster Labels,Latitude,Longitude,Venue_name,Venue_lat,Venue_long,Venue_cat
73,M4T,Central Toronto,Moore Park / Summerhill East,0.032787,1,43.689574,-79.383160,Second Cup,43.689814,-79.394811,Coffee Shop
72,M4T,Central Toronto,Moore Park / Summerhill East,0.032787,1,43.689574,-79.383160,Starbucks,43.686152,-79.392874,Coffee Shop
71,M4T,Central Toronto,Moore Park / Summerhill East,0.032787,1,43.689574,-79.383160,Second Cup,43.688257,-79.393778,Coffee Shop
84,M4W,Downtown Toronto,Rosedale,0.047619,1,43.679563,-79.377529,Tim Hortons,43.672155,-79.376993,Coffee Shop
82,M4W,Downtown Toronto,Rosedale,0.047619,1,43.679563,-79.377529,Starbucks,43.671082,-79.380756,Coffee Shop
83,M4W,Downtown Toronto,Rosedale,0.047619,1,43.679563,-79.377529,Aroma Espresso Bar,43.672154,-79.377885,Coffee Shop
70,M4T,Central Toronto,Moore Park / Summerhill East,0.032787,1,43.689574,-79.383160,Starbucks,43.688467,-79.393801,Coffee Shop


In [350]:
cluster1.groupby(['Borough','Neighborhood','Venue_cat']).count()

,,,Postal Code,Playground,Cluster Labels,Latitude,Longitude,Venue_name,Venue_lat,Venue_long
Borough,Neighborhood,Venue_cat,,,,,,,,
Central Toronto,Moore Park / Summerhill East,Coffee Shop,4,4,4,4,4,4,4,4
Downtown Toronto,Rosedale,Coffee Shop,3,3,3,3,3,3,3,3


In [351]:
#Cluster 2
cluster2 = playarea.loc[playarea['Cluster Labels'] == 2]
print(cluster2.shape[0])
cluster2

24


,Postal Code,Borough,Neighborhood,Playground,Cluster Labels,Latitude,Longitude,Venue_name,Venue_lat,Venue_long,Venue_cat
263,M5R,Central Toronto,The Annex / North Midtown / Yorkville,0.010000,2,43.672710,-79.405678,Herbs & Nutrition Qi Natural Food,43.665013,-79.411898,Health Food Store
267,M5R,Central Toronto,The Annex / North Midtown / Yorkville,0.010000,2,43.672710,-79.405678,B Espresso Bar,43.668143,-79.396087,Coffee Shop
266,M5R,Central Toronto,The Annex / North Midtown / Yorkville,0.010000,2,43.672710,-79.405678,Starbucks,43.668615,-79.397387,Coffee Shop
265,M5R,Central Toronto,The Annex / North Midtown / Yorkville,0.010000,2,43.672710,-79.405678,Volta Coffee,43.667314,-79.412145,Coffee Shop
264,M5R,Central Toronto,The Annex / North Midtown / Yorkville,0.010000,2,43.672710,-79.405678,Kimpton St. George,43.667368,-79.400627,Hotel
262,M5R,Central Toronto,The Annex / North Midtown / Yorkville,0.010000,2,43.672710,-79.405678,Creeds Coffee Bar,43.674100,-79.410838,Coffee Shop
330,M6H,West Toronto,Dufferin / Dovercourt Village,0.014286,2,43.669005,-79.442259,Balzac’s Coffee,43.670042,-79.448342,Coffee Shop
361,M6R,West Toronto,Parkdale / Roncesvalles,0.010000,2,43.648960,-79.456325,Outpost Coffee Roasters,43.656059,-79.454106,Coffee Shop
364,M6R,West Toronto,Parkdale / Roncesvalles,0.010000,2,43.648960,-79.456325,Qi Natural Food,43.645817,-79.448810,Food & Drink Shop
363,M6R,West Toronto,Parkdale / Roncesvalles,0.010000,2,43.648960,-79.456325,Lit Espresso Bar,43.645846,-79.448830,Coffee Shop


In [352]:

cluster2.groupby(['Borough','Neighborhood','Venue_cat']).count()

Postal Code  \
Borough          Neighborhood                          Venue_cat                        
Central Toronto  The Annex / North Midtown / Yorkville Coffee Shop                  4   
                                                       Health Food Store            1   
                                                       Hotel                        1   
Downtown Toronto Christie                              Coffee Shop                  6   
                                                       Health Food Store            1   
West Toronto     Dufferin / Dovercourt Village         Coffee Shop                  6   
                 Parkdale / Roncesvalles               Coffee Shop                  4   
                                                       Food & Drink Shop            1   

                                                                          Playground  \
Borough          Neighborhood                          Venue_cat                       
Central Toronto  The Annex / North Midtown / Yorkville Coffee Shop                 4   
                                                       Health Food Store           1   
                                                       Hotel                       1   
Downtown Toronto Christie                              Coffee Shop                 6   
                                                       Health Food Store           1   
West Toronto     Dufferin / Dovercourt Village         Coffee Shop                 6   
                 Parkdale / Roncesvalles               Coffee Shop                 4   
                                                       Food & Drink Shop           1   

                                                                          Cluster Labels  \
Borough          Neighborhood                          Venue_cat                           
Central Toronto  The Annex / North Midtown / Yorkville Coffee Shop                     4   
                                                       Health Food Store               1   
                                                       Hotel                           1   
Downtown Toronto Christie                              Coffee Shop                     6   
                                                       Health Food Store               1   
West Toronto     Dufferin / Dovercourt Village         Coffee Shop                     6   
                 Parkdale / Roncesvalles               Coffee Shop                     4   
                                                       Food & Drink Shop               1   

                                                                          Latitude  \
Borough          Neighborhood                          Venue_cat                     
Central Toronto  The Annex / North Midtown / Yorkville Coffee Shop               4   
                                                       Health Food Store         1   
                                                       Hotel                     1   
Downtown Toronto Christie                              Coffee Shop               6   
                                                       Health Food Store         1   
West Toronto     Dufferin / Dovercourt Village         Coffee Shop               6   
                 Parkdale / Roncesvalles               Coffee Shop               4   
                                                       Food & Drink Shop         1   

                                                                          Longitude  \
Borough          Neighborhood                          Venue_cat                      
Central Toronto  The Annex / North Midtown / Yorkville Coffee Shop                4   
                                                       Health Food Store          1   
                                                       Hotel                      1   
Downtown Toronto Christie                              Coffee Shop                6  

In [ ]:


num1 =cluster0[cluster0['Venue_cat']=='playground']
print("There are {} playground in Cluster 0".format(len(num1)))

num2 =cluster1[cluster1['Venue_cat']=='playground']
print("There are {} playground in Cluster 1".format(len(num2)))

num3 =cluster2[cluster2['Venue_cat']=='playground']
print("There are {} playground in Cluster 2".format(len(num3)))